#Máquinas de Boltzmann Restringidas

# Instalamos pytorch

In [1]:
#pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Clonamos el repositorio para obtener el dataset

In [2]:
!git clone https://github.com/joanby/deeplearning-az.git

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 10153 (delta 25), reused 39 (delta 16), pack-reused 10096
Receiving objects: 100% (10153/10153), 236.95 MiB | 15.74 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Checking out files: 100% (10108/10108), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importar las librerías

In [1]:
# Haremos un sistema de recomendación como el que podría utilizar Netflix o algun supermercado en la recomendación de productos
# En este caso haremos un sistema que prediga si a un usuario le gustará o no una película con RBM, y con AE haremos uno que prediga la calificación del usuario a la película (ambas sobre las películas que no vieron) (el primero tendrá una salida binaria, mientras que el segundo usará una clasificación de salida del 1-5, con una predicción más sólida)
import numpy as np
import pandas as pd
import torch
import torch.nn as nn # para crear redes neuronales
import torch.nn.parallel # para hacer calculos en paralelo
import torch.optim as optim # para minimizar nuestro error
import torch.utils.data
from torch.autograd import Variable

# Importar el dataset

In [2]:
# Para RBM y AE usaremos el mismo dataset, uno de valoraciones de clientes sobre películas
# Hay un PDF en la carpeta que explica más a detalle la teoría y práctica de RBM (recordar que queremos minimizar la energía de un estado del sistema, que es lo mismo que maximizar la probabilidad del mismo con Divergencia Contrastante)
# Cargamos, del dataset de 1 millón de valoraciones: las películas, usuarios y puntajes
movies = pd.read_csv("ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1') # así se guardan correctamente los df cuando vienen en formato .dat (header=None cuando en el archivo no hay encabezado de las columnas y ver encoding que se usó para realizar el dataset)
users  = pd.read_csv("ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# Sobre ratings particularmente es que vamos a dividir en train/test, ya que es en base a este dataset que vamos a crear el algoritmo, los otros son para luego tener una referencia

In [3]:
print(movies) # hay 3882 películas. El identificador de la película está en la primera columna, en la segunda el nombre y en la tercera su género/categoría
print('---------------------------------------------------------------------------')
print(users) # hay 6040 usuarios. El identificador de los usuarios está en la primera columna, en la segunda su género, la tercera es la edad, la cuarta indica la ocupación del usuario y la quinta es el código postal de las personas
print('---------------------------------------------------------------------------')
print(ratings) # hay 1000209 valoraciones. La primera columna indica el identificador del usuario que emitió la valoración, la segunda columna sobre qué película emitió la valoración (identificador de la película), la tercera el puntaje/valoración otorgado a la película (1-5) y la cuarta es una referencia temporal sobre cuando fue que se emitió la valoración (poco importante)

         0                                   1                             2
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
...    ...                                 ...                           ...
3878  3948             Meet the Parents (2000)                        Comedy
3879  3949          Requiem for a Dream (2000)                         Drama
3880  3950                    Tigerland (2000)                         Drama
3881  3951             Two Family House (2000)                         Drama
3882  3952               Contender, The (2000)                Drama|Thriller

[3883 rows x 3 columns]
---------------------------------------------------

# Preparar el conjunto de entrenamiento y el conjunto de testing

In [4]:
# Vamos a tomar para esto el set de datos de 100.000 valoraciones, en donde hay 5 conjuntos de entrenamiento (u.base) y 5 conjuntos de test (u.test) para hacer k-fold cross validation
# Esto nos servirá para entrenar hacia la NN, pero nosotros no usaremos validación cruzada, sino que sólo tomaremos u1.base y u1.test
training_set = pd.read_csv("ml-100k/u1.base", sep = "\t", header = None) # para transformar en df usamos tabulador como separador (como no tiene nombre de columna el dataset le ponemos header=None)
training_set = np.array(training_set, dtype = "int") # para Pytorch necesito arrays
test_set = pd.read_csv("ml-100k/u1.test", sep = "\t", header = None)
test_set = np.array(test_set, dtype = "int")
# training_set incluye el 80% del conjunto de 100.000 valoraciones (usuarios, películas, valoración y referencia de tiempo), mientras que test_set el 20% restante (con misma disposición de columnas)
# Este dataset pequeño de 100k lo usaremos para crear la Máquina de Boltzmann Reducida, intentando predecir si al usuario le gustará o no una película no vista

# Obtener el número de usuarios y de películas

In [5]:
# Necesitamos obtener el número de usuarios y películas, para convertir el set de train y test en matrices, donde las filas serán los usuarios, las columnas las películas y las celdas las valoraciones de los usuarios sobre cada películas
# Tanto para k-fold como para div por porcentajes, tendríamos que encontrar el máximo en los conjuntos de entrenamiento y de test que sean mayores a los otros conjuntos
nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0]))) # el num de usuario más grande del conjunto de train comparado con el más grande de test, quedándome con el más grande de ambos (usuarios en columna 0)
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1]))) # el num de película más grande del conjunto de train comparado con el más grande de test, quedándome con el más grande de ambos (películas en columna 1)
# Si hicieramos k-fold, agregar los otros 4 conjuntos en cada fila

print(nb_users) # existen 943 usuarios (tomando train y test)
print('----')
print(nb_movies) # existen 1682 películas (tomando train y test)

943
----
1682


# Convertir los datos en un array X[u,i] con usuarios u en fila y películas i en columna

In [6]:
# Necesitamos esta estructura de datos para BM y AE
def convert(data): # hacemos una función para hacer que las filas sean los usuarios y las columnas películas (conjunto de datos a transformar)
    new_data = [] # matriz final (lo hago como lista de listas)
    for id_user in range(1, nb_users+1): # itero sobre los usuarios (empieza desde el 1 incluyendo el último con el +1) 
        id_movies = data[:, 1][data[:, 0] == id_user] # me quedo con los id de las películas valoradas por ese usuario (segunda columna) (usamos el segundo corchete para filtrar que pertenezca el id de la película al usuario actual en el for)
        id_ratings = data[:, 2][data[:, 0] == id_user] # me quedo con los puntajes de valoración realizados por ese usuario (tercera columna) (usamos el segundo corchete para filtrar que pertenezca la valoración al usuario actual en el for)
        ratings = np.zeros(nb_movies) # inicializo una lista de 0 con las valoraciones del usuario del tamaño del número de películas total del dataset (suponemos que no vió ninguna inicialmente)
        ratings[id_movies-1] = id_ratings # agregamos las valoraciones del usuario en ratings guiándonós de los índices de las películas -1 (que van del 1 al 1682), para que la película 1 vaya en la posición 0 de nuestra lista (y también incluya a la última película)
        new_data.append(list(ratings)) # empalmamos la información de cada usuario (ratings) en la nueva matriz que recopile todos los usuarios (new_data)
    return new_data # devolvemos el nuevo conjunto de datos
# Hay 1682 columnas pertenecientes a la cantidad de usuarios que querramos analizar sus valoraciones (en caso de que las halla visto del 1-5, sino le pongo un 0)

In [7]:
# Convertimos al set de entrenamiento y test en la matriz con filas de usuarios y columnas de películas (mostrar el set transformado requiere mucha RAM por la cantidad de columnas y filas) 
# (Como si fuera un array de un csv)
training_set = convert(training_set)
test_set = convert(test_set)


# Convertir los datos a tensores de Torch

In [8]:
# Tensores son matrices que contienen elementos de un solo tipo de dato (matriz multidimensional). Debemos transformarlas para ingresar a la RBM
training_set = torch.FloatTensor(training_set) # lo convertimos en un tensor de float
test_set = torch.FloatTensor(test_set)

# Convertir las valoraciones a valores binarios 1 (Me gusta) o 0 (No me gusta)

In [9]:
# Como en RBM vamos a trabajar solo con 'Me Gusta' y 'No Me gusta' (valoraciones binarias), adaptamos las valoraciones originales de los usuarios que han emitido del 1-5 y ponemos un -1 sobre las no valoradas
training_set[training_set == 0] = -1 # a las películas no visualizadas por el usuario le asgnamos un valor -1 (acordarse que inicialmente le habiamos puesto un 0 a las no vistas por el usuario)
training_set[training_set == 1] = 0 # Pytorch no acepta un OR en la condición
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1 # a las valoraciones 3, 4 y 5 le asignamos un 1 (Me gusta), mientras que a las 1 y 2 le asignamos un 0 (No me gusta)
# Si queremos brindar una salida binaria (predecir si al usuario le gustará (1) o no (0) una película, debemos tener entradas del mismo tipo)

In [10]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Crear la arquitectura de la Red Neuronal (Modelo Probabilistico Gráfico)

In [11]:
# CON ESPECIFICAR EL NÚMERO DE NODOS VISIBLES Y OCULTOS PODEMOS ENTRENAR CUALQUIER RBM (USANDO ESTA FUNCIÓN)
# Recordar que es un modelo No Supervisado (no tenemos etiquetas sobre la variable a predecir, sino que es más difusa y particular la predicción)
# Creamos una clase (POO) de la arquitectura RBM, conectando los nodos ocultos con un Modelo Probabilístico Gráfico (conecto en forma de grafo) (creamos objetos de esa clase particular)
class RBM(): # a partir de una distribución de Bernoulli, que predice un resultado binario (éxito o fracaso)
    
    def __init__(self, nv, nh): # constructor, para inicializar la RBM (self: guardarmos aquí las variables locales para luego usarlas, nv: num de nodos visibles, nh: num de nodos ocultos). El método __init__ inicializará los pesos de las capas conectando todos con todos (incluso también con el término independiente)
        self.W = torch.randn(nh, nv) # inicializo pesos (que relacionan un nodo de la capa visible con uno de la capa oculta con valores al azar pequeño pero no nulos, que se distribuyan normal estándar con randn). Tensor (matriz) de tamaño nh x nv
        self.a = torch.randn(1, nh) # inicializo términos independientes (sesgo: el valor del nodo oculto conociendo el visible) de los nodos ocultos. Vector 1 x nh con inicialización random dist normal estandarizada (1° dim con num de elementos a entrenar y 2° dim correspondiente al sesgo, aclaramos dim adicional)
        self.b = torch.randn(1, nv) # inicializo términos independientes de los nodos visibles de la misma forma con el tensor bidimensional al igual que arriba
    
    # Como vamos a tomar en x mini_batches (entrenamiento por bloques), requiero expansión de los términos independientes acorde al tamaño del mismo (#wx = mini_batch_size x nh, #x = mini_batch_size x nv)
    # A partir de los valores de las capas visibles, vemos si se activan o no los nodos de las capas ocultas (con muestreo de Gibbs para converger rápidamente)
    # x: entradas, relacionadas a las películas
    def sample_h(self, x): # muestreo/calculo de las probabilidades de activación de los nodos de la capa oculta, conociendo los valores de los nodos visibles (función de activación sigmoidea) (con muestro de Gibbs para aproximar los gradientes de prob) (x: los valores de la capa visible, valoraciones de usuario)         
        wx = torch.mm(x, self.W.t()) # para cada nodo oculto voy a calcular la prob de que se active, para luego con esa prob realizar un muestreo con prob de Bernoulli (para ver si lo activaría o no)(mm: matrix multiplication y t para tomar la transpuesta de los pesos)
        activation = wx + self.a.expand_as(wx) # Obtenemos w.x + a(pesos por neuronas + el sesgo de la capa oculta), expandimos al tamaño del mini_batch
        p_h_given_v = torch.sigmoid(activation) # Aplicamos la función sigmoide que activará el nodo oculto (obteniendo probabilidades entre 0 y 1 de que el nodo se active según los nodos de entrada) (prob de activarse nodos ocultos relacionados con los favoritismos del usuario serán elevadas)
        return p_h_given_v, torch.bernoulli(p_h_given_v) # devolvemos no solo las probabilidades de los nodos ocultos conociendo los visibles, sino también una muestra Bernoulli con respecto a los nodos ocultos, que me mostrará cuales son los que activará (genero un vector con 0 y 1 a partir de que cada nodo oculto se active o no)

    # (tamaños: #wy = mini_batch_size x nv, #y = mini_batch_size x nh)
    # A partir de los valores (prob de activaciones) de los nodos ocultos, vemos si se activan o no se activan los valores de la capa visible (otro muestreo de Gibbs para converger rápidamente)
    # y: lo que viene de la capa oculta
    def sample_v(self, y): # muestreo de las probabilidades de los nodos de la capa visible, para luego concatenarlos a partir de los nodos de la capa oculta (prob condicionada: sabiendo los nodos de la capa visible, cuales son los de la oculta que influyen y viceversa relación)           
        wy = torch.mm(y, self.W) # no hace falta tomar la transpuesta de los pesos
        activation = wy + self.b.expand_as(wy) # sumamos los términos independientes de la capa visible expandidos acorde al mini_batch (entrenamiento por bloques)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h) # vemos cuales de los nodos visibles deben activarse con Bernoulli (muestra aleatoria) dadas las prob de activaciones de los nodos ocultos (que películas recomendar dadas las activaciones de los nodos ocultos para cada usuario, infiriendo si le gustará o no)

# Usamos Divergencia Contrastante para aproximar el gradiente del logaritmo de la probabilidad (optimizar pesos para maximizar probabilidad del estado) 
# En vez de seguir la dirección del gradiente, pasando a través de diversos estados, aproximamos para que el estado actual sea el de mayor probabilidad con CD (ajustando los pesos en tal dirección con cadena de Gibbs en k-pasos de CD)        
# Creamos el método de entrenamiento del algoritmo    
    def train(self, v0, vk, ph0, phk): # v0: calificación original que han dado los usuarios (nodos visibles originales), vk: nodos visibles despues de k-pasos (despues de k idas-vueltas de CD), ph0: probabilidades en la primera iteración de los nodos ocultos dados los nodos visibles originales, phk: probabilidades de los nodos ocultos despues de k-iteraciones de CD  
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t() # actualizamos los pesos (tomando el valor antiguo de los pesos y agregándole el producto de la prob de activación de v0 * ph0 - el producto de vk * phk). Primera actualización de la CD que actualiza los valores de los pesos
        self.b += torch.sum((v0 - vk), 0) # actualizamos los sesgos (t.i)  de nodos visibles (valoración inicial del usuario - valoración que el algoritmo decide despues de k-iteraciones). Agregamos el 0 para mantener el formato del tensor en 2D y no nos salte error. Ajustamos los sesgos visibles con los valores visibles modificados
        self.a += torch.sum((ph0 - phk), 0) # actualizamos los sesgos (t.i) con prob de act de nodos ocultos dados los nodos visibles (prob de que se activen los oscultos sabiendo la prob del original - prob de que se activen los ocultos conocidos los valores de la iteración k-ésima). Ajustamos los sesgos ocultos con las restas de las prob de los ocultos dados los visibles. Agregamos el 0 para mantener tensor 2D

# Creamos la RBM

In [12]:
# Inicialmente para crear la RBM, solo necesitamos saber el número de nodos visibles y el número de nodos ocultos
nv = len(training_set[0]) # nodos visibles: cantidad de columnas en el dataset (películas)
nh = 100 # nodos ocultos: a determinar (de 1682 nodos visibles, tenemos que decirle a la RBM la cantidad de correlaciones/características que encuentre) (probamos con 100, si vemos poca capacidad de predicción aumentamos con GridSearch)
batch_size = 100 # tamaño del bloque de aprendizaje (actualizaremos los pesos luego de cada bloque introducido) (si ponemos 1 se actualiza dato a dato, mientras mayor sea menos tarda la convergencia, podemos usar tmb GridSearch)

rbm = RBM(nv, nh) # creamos un objeto de la clase con nuestros valores

# Entrenar la RBM


In [13]:
nb_epoch = 10 # definimos el número de épocas (como el número de datos no es muy elevado y la predicción es binaria, ponemos pocas) (podemos subirla si no vemos buenos resultados)
for epoch in range(1, nb_epoch+1): # for para cada época 
    training_loss = 0 # inicializamos la variable para cuantificar la pérdida de la función de pérdida que mida el error (usaremos RSME para AE y MAE, por tener solo 0 y 1, para RBM) 
    s = 0. # contador de observaciones (debremos normalizar el error por el número total de obs predichas, sacando el error relativo y asignándole así contexto al mismo) (ponemos 0. para inicializarlo como decimal)
    # Comenzamos el entrenamiento real (llamando al sample_h, sample_v y train cuando tengamos un usuario/bloque de usuarios para tener neuronas activadas de la capa oculta, luego en base a ellas las activadas de la capa visible y entrenar en base a ese resultado)
    # CD solo necesita la valoración inicial y la de luego de k-pasos, y las prob de activar un nodo oculto al principio y luego de k-pasos (en nuestro caso los usuarios los necesitamos en bloques)
    for id_user in range(0, nb_users - batch_size, batch_size): # for para recorrer a los usuarios en bloques (va del 0 hasta el dato con el q se puede tomar el último bloque avanzando según el tamaño del batch_size) (si lo hicieramos de 1 en 1 no agregar la 3ra parte del for)
        vk = training_set[id_user:id_user+batch_size] # tomamos las valoraciones de los 100 usuarios del bloque para entrenar (a estas valoraciones las iremos actualizando a medida que apliquemos el muestro de Gibbs en k-pasos)
        v0 = training_set[id_user:id_user+batch_size] # tomamos las valoraciones de los 100 usuarios del bloque para entrenar (las valoraciones iniciales siempre serán las mismas) (las usaremos para contrastar el error de vk respecto a las valoraciones iniciales)
        ph0,_ = rbm.sample_h(v0) # prob de q se active un nodo oculto conociendo las entradas (valoraciones) originales (usamos sample_h para que nos devuelva prob de activar nodo oculto a partir de valores de entrada originales, v0) (ponemos ,_ para indicar que el segundo parámetro de return no me interesa)
        # Lo que hace el algoritmo es un muestreo de Gibbs (camino ida y vuelta de los nodos visibles a los ocultos y viceversa). Intentaremos en cada ida y vuelta acercarnos a las buenas clasificaciones que queremos pronosticar
        # Iremos activando nodos ocultos según prob Bernoulli
        for k in range(10): # bucle que ejecute la cadena de Gibbs durante k-pasos del muestreo aleatorio o 'random walk' (elijo 10 iteraciones aleatorias del algoritmo)
            _,hk = rbm.sample_h(vk) # buscamos en este caso el segundo parámetro (nodos ocultos que se activen), pasándole los valores del paso inmediatamente anterior (vk) (los nodos ocultos que se activan en el paso k-ésimo de la CD)
            _,vk = rbm.sample_v(hk) # a partir de los nodos ocultos que se acaban de activar debo actualizar los nodos visibles. Así culminaría la primera ida y vuelta del muestreo de Gibbs hacia la convergencia (hasta k=10)
            # a partir del la activación de los nodos ocultos en k(10) pasos del muestreo de Gibbs, calculará las activaciones de los nodos visibles, tendiendo a la convergencia de la calificación predicha por el algoritmo
            vk[v0 < 0] = v0[v0 < 0] # congelamos (mantenemos los valores iniciales) los nodos visibles que originalmente valían -1 (películas no vistas) para que no sean actualizados. Los necesitaré predecir al final del algoritmo, ahora estamos entrenando nodos ocultos con la información que ya conocemos (películas vistas)
        phk,_ = rbm.sample_h(vk) # sacamos el phk tomando vk (últimos valores conocidos de los nodos visibles luego de k-pasos) que nos falta para actualizar pesos (usar función train)
        rbm.train(v0, vk, ph0, phk) # actualizamos los term indeps y pesos (valoraciones originales de usuarios, valoraciones luego de k pasos de CD, prob iniciales de act de nodos ocultos y prob de act de nodos ocultos luego de k-pasos de CD).
        # Así se maximizará la prob, asignándole más pesos a los nodos ocultos relevantes para las clasificaciones de películas, acercándonos a las clasificaciones reales hechas por los usuarios respecto a las películas vistas
        # Incrementamos el error tomando (MAE para clas binaria) la media absoluta entre la diferencia de la calificación original (v0 cuando no sea -1, solo las vistas) con respecto calificación predicha luego de k-pasos (vk cuando no sea -1, solo las vistas) 
        training_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) # cuanto de adecuación a las valoraciones iniciales estamos ganando por época (medida del error) (vemos si los nuevos pesos me dan valores dif de nodos visibles activados, aproximándose a los originales)
        s += 1. # actualizamos el contador de observaciones, para obtener luego el error (MAE) relativo/normalizado 
    print("Epoch: "+str(epoch)+", Loss: "+str(training_loss/s)) # mostramos la época por la que vamos y el valor de la función de pérdida normalizado (contrarrestando el efecto de las iteraciones, para que no crezca con cada ejecucuón acumulándolo, sino yéndolo dividiendo por el num de obs)
# Como vemos, el valor de la función de pérdida tiende a 0.25, lo que significa que nuestro algoritmo se ajusta al set de entrenamiento logrando predecir correctamente 3 de cada 4 observaciones

Epoch: 1, Loss: tensor(0.3402)
Epoch: 2, Loss: tensor(0.2320)
Epoch: 3, Loss: tensor(0.2500)
Epoch: 4, Loss: tensor(0.2494)
Epoch: 5, Loss: tensor(0.2487)
Epoch: 6, Loss: tensor(0.2484)
Epoch: 7, Loss: tensor(0.2475)
Epoch: 8, Loss: tensor(0.2483)
Epoch: 9, Loss: tensor(0.2477)
Epoch: 10, Loss: tensor(0.2458)


# Testear la RBM

In [14]:
# Vemos si nuestro modelo se comporta bien prediciendo para otros usuarios, y cuantificamos también la pérdida respecto a estos nuevos datos (comparar con rendimiento en train para ver si se produce Overffiting o Underffiting)
# En MCMC (Monte Carlo Markov Chain) la valoración se va acercando a su valor real
# No hay entrenamiento, por lo que no requerimos las épocas ni un bucle sobre ellas, ya que la predicción será solo 1 (sobre el conjunto de test) (Shift + Tab para indexar todo)
testing_loss = 0 # pérdida en conjunto de test
s = 0.
for id_user in range(nb_users): # podemos eliminar el bucle que recorre por bloques, para predecir de 1 en 1 observación (elimino el batch_size como 3er elemento) (el bucle pasa por cada usuario)
    v = training_set[id_user:id_user+1] # tomamos la valoración del individuo actual (estado inicial de la predicción) (mantenemos el conjunto de train, ya que el sistema comenzará con las entradas conocidas)
    vt = test_set[id_user:id_user+1] # tomamos la valoración del individuo actual (estado de la predicción sobre las películas no vistas (-1) del conjunto de train luego de t-pasos a lo largo del tiempo, convergiendo a la pred de la RBM)
    # MCMC tiene en cuenta que las prob no son las mismas (caminata aleatoria, que nos basta con dar solo 1 paso para elaborar una predicción aproximada, ya que hemos entrenado al algoritmo para que tienda a adecuarse a los datos) (un solo viaje de ida y vuelta con el muestreo de Gibbs)
    if len(vt[vt>=0]) > 0: # bucle que pasa por todas las valoraciones de los usuarios de test, siempre que tengan por lo menos vista 1 película (para saber preferencias)
        _,h = rbm.sample_h(v) # en un sólo paso, calculo los nodos ocultos que se activarían en base a los visibles del usuario (películas vistas y valoradas por el mismo)
        _,v = rbm.sample_v(h) # volveríamos para ver los nodos visibles que se activan dados los nodos ocultos activados para el usuario
        # Podemos agregar las 2 líneas de código anterior para dar otro paso en MCMC
        testing_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # actualizamos la función de pérdida, midiendo el error a partir de las diferencias entre las valoraciones que ya conocemos del usuario y las predichas por el modelo (no podemos medir el error con respecto a las películas que el usuario no vió, ya que no sabriamos su resultado)
        s += 1. # normalizar el error de predicción
        print("Testing Loss: "+str(testing_loss/s)) # pérdida para cada usuario (efectividad en la predicción del RBM tomando al train)

# Aca vemos el error de la predicción con respecto a las películas que el usuario ya vió (suponemos que será el mismo con respecto a las películas no vistas)

Testing Loss: tensor(0.2044)
Testing Loss: tensor(0.2158)
Testing Loss: tensor(0.2593)
Testing Loss: tensor(0.2445)
Testing Loss: tensor(0.2789)
Testing Loss: tensor(0.2638)
Testing Loss: tensor(0.2509)
Testing Loss: tensor(0.2411)
Testing Loss: tensor(0.2254)
Testing Loss: tensor(0.2195)
Testing Loss: tensor(0.2223)
Testing Loss: tensor(0.2198)
Testing Loss: tensor(0.2289)
Testing Loss: tensor(0.2289)
Testing Loss: tensor(0.2454)
Testing Loss: tensor(0.2389)
Testing Loss: tensor(0.2575)
Testing Loss: tensor(0.2503)
Testing Loss: tensor(0.2476)
Testing Loss: tensor(0.2512)
Testing Loss: tensor(0.2562)
Testing Loss: tensor(0.2594)
Testing Loss: tensor(0.2516)
Testing Loss: tensor(0.2488)
Testing Loss: tensor(0.2400)
Testing Loss: tensor(0.2411)
Testing Loss: tensor(0.2445)
Testing Loss: tensor(0.2411)
Testing Loss: tensor(0.2409)
Testing Loss: tensor(0.2421)
Testing Loss: tensor(0.2443)
Testing Loss: tensor(0.2448)
Testing Loss: tensor(0.2465)
Testing Loss: tensor(0.2540)
Testing Loss: 

In [1]:
# Si quisieramos usar RMSE en vez de DISTANCIA MEDIA para medir rendimiento, deberíamos cambiar la función de pérdida en train a esto:
# train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2))
# y en test a esto:
# test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) 
# Usando el RMSE, nuestra MBR obtendría un error alrededor de 0.46. Pero ten cuidado, aunque parece similar, no se debe confundir el RMSE y la distancia promedio. Un RMSE de 0.46 no significa que la distancia promedio entre la predicción y la verdad del dataset sea 0.46. En modo aleatorio terminaríamos con un RMSE de alrededor de 0.72. Un error de 0.46 corresponde al 75% de la predicción exitosa.

In [7]:
# Así se comprueba que un 0.25 de Distancia Media del modelo se corresponde con un 75% de éxito aproximado en la predicción
import numpy as np
u = np.random.choice([0,1], 100000)
v = np.random.choice([0,1], 100000)
u[:50000] = v[:50000]
print(sum(u==v)/float(len(u))) # -> obtendremos 0.75
print(np.mean(np.abs(u-v))) # -> obtendremos 0.25

0.74763
0.25237


In [17]:
# En vt tendríamos las predicciones del modelo con respecto a las películas no vistas por cada usuario, las cuales servirán para recomendación
torch.set_printoptions(precision=2, threshold=torch.inf) # establecer opciones de impresión: precisión de 2 decimales y mostrar el tensor completo
vt
# Probar usar predict también

tensor([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1.